In [1]:
import sys
sys.path.append("../../src")

import numpy as np
import pandas as pd

import lightgbm

from utils import ndcg_calculator

In [42]:
test_answer_week = pd.read_parquet("../../data/test_answer_week.parquet")
test_answer_month = pd.read_parquet("../../data/test_answer_month.parquet")

df_train_week = pd.read_parquet("../../data/train_week.parquet")
df_train_month = pd.read_parquet("../../data/train_month.parquet")

sample_sumbission_week = pd.read_parquet("../../data/sample_sumbission_week.parquet")
sample_sumbission_month = pd.read_parquet("../../data/sample_sumbission_month.parquet")

meta_df = pd.read_csv("all_meta_df.csv")
profile_df = pd.read_csv("../../data/profile_data.csv")


df_train_week.sort_values(by='log_dt', inplace=True)
df_train_month.sort_values(by='log_dt', inplace=True)

/var/folders/wz/2dv1p69j3pv2c66bj9jt_hbc0000gn/T/ipykernel_94634/470544365.py:10: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  meta_df = pd.read_csv("all_meta_df.csv")


# preprocess

In [61]:
n = 25

In [117]:
# purchase top
purchase_week_df = df_train_week[df_train_week['payment']>0]
purchase_week_df.drop_duplicates(subset=['profile_id','album_id'], inplace=True)

purchase_top_idxs = purchase_week_df.album_id.value_counts().index
sample_sumbission_week['album_id'] = sample_sumbission_week['album_id'].apply(lambda x: list(purchase_top_idxs)[:25])

In [125]:
# MP
MP_list = df_train_week.album_id.value_counts().index
sample_sumbission_week['album_id'] = sample_sumbission_week['album_id'].apply(lambda x: list(MP_list)[:25])

In [126]:
ndcg_calculator(test_answer_week, sample_sumbission_week, 25)

0.0357749766813933

In [131]:
MP_candidate = df_train_week[df_train_week['album_id'].isin(MP_list[:300])]

,profile_id,ss_id,act_target_dtl,album_id,payment,continuous_play,short_trailer,log_dt,log_date
899162,25844,20220301000456,MKID003,1881,NaN,N,N,2022-03-01 00:05:00,2022-03-01
899161,25844,20220301000456,MKID003,1881,NaN,N,N,2022-03-01 00:05:00,2022-03-01
899163,25844,20220301000456,MKID003,4608,NaN,N,N,2022-03-01 00:06:00,2022-03-01
899164,25844,20220301000456,MKID003,4608,NaN,N,N,2022-03-01 00:06:00,2022-03-01
109724,2794,20220301000888,MKID003,2641,NaN,N,N,2022-03-01 00:08:00,2022-03-01
...,...,...,...,...,...,...,...,...,...
409585,9671,20220423235615,MKID003,56,NaN,Y,N,2022-04-23 23:59:00,2022-04-23
409584,9671,20220423235615,MKID003,0,NaN,Y,N,2022-04-23 23:59:00,2022-04-23
724845,19290,20220423235128,MKID003,2285,NaN,Y,N,2022-04-23 23:59:00,2022-04-23
724844,19290,20220423235128,MKID003,2285,NaN,Y,N,2022-04-23 23:59:00,2022-04-23


In [ ]:
def lgbm_preprocess(train, valid):
    X_train = train.drop(columns=['rating'])
    y_train = train['rating']
    
    X_validation = valid.drop(columns=['rating'])
    y_validation = valid['rating']
    
    qids_train = train.groupby('profile_id')['profile_id'].count().to_numpy()
    qids_validation = valid.groupby('profile_id')['profile_id'].count().to_numpy()
    
    return X_train, y_train, X_validation, y_validation, qids_train, qids_validation

In [ ]:
cf_df = cf_df.sample(frac=1, random_state=42).reset_index(drop=True)

# model

In [ ]:
model_params = {
    'n_estimators':1,
    'verbose':1,
    'random_state':42,
    'eval_at':25
}

In [ ]:
model = lightgbm.LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators= model_params['n_estimators'],
    importance_type='gain',
    verbose= model_params['verbose'],
    random_state= model_params['random_state']
)

In [ ]:
model.fit(
    X=X_train,
    y=y_train,
    group=qids_train,
    eval_set=[(X_validation, y_validation)],
    eval_group=[qids_validation],
    eval_at= model_params['eval_at'],
    verbose= model_params['verbose'],
)

In [28]:
# from datetime import timedelta
# import math

# def day_week_numbers(dates:pd.Series):
#     pd_dates = pd.to_datetime(dates)

#     unique_dates = pd.Series(pd_dates.unique())
#     numbered_days = unique_dates - unique_dates.min() + timedelta(1)
#     numbered_days = numbered_days.dt.days
#     extra_days = numbered_days.max() % 7
#     numbered_days -= extra_days
#     day_weeks = (numbered_days / 7).apply(lambda x: math.ceil(x))
#     day_weeks_map = pd.DataFrame(
#         {"day_weeks": day_weeks, "unique_dates": unique_dates}
#     ).set_index("unique_dates")["day_weeks"]
#     all_day_weeks = pd_dates.map(day_weeks_map)
#     all_day_weeks = all_day_weeks.astype("int8")
    
#     return all_day_weeks

# def day_numbers(dates: pd.Series):
#     """
#     assign consecutive number to dates, such that earliest date is 0
#     args:
#     dates: pd.Series of date strings
#     returns:
#     all_day_numbers: pd.Series of day numbers each day in original pd.Series corresponds to
#     """

#     unique_dates = dates.unique()
# #     unique_dates = unique_dates.to_pandas()
#     unique_dates = np.sort(unique_dates)
#     number_range = np.arange(len(unique_dates))
#     date_number_dict = dict(zip(unique_dates, number_range))

#     all_day_numbers = dates.apply(date_number_dict)
#     all_day_numbers = all_day_numbers.astype("int16")

#     return all_day_numbers

In [29]:
all_day_weeks = day_week_numbers(df_train_week['log_date'])

In [30]:
all_day_numbers = day_numbers(df_train_week['log_date'].astype(str))

AssertionError: 

In [20]:
df_train_week['log_date'].apply(lambda x: )

899160   2022-03-01
899162   2022-03-01
899161   2022-03-01
899163   2022-03-01
899164   2022-03-01
            ...    
58990    2022-04-23
724845   2022-04-23
724844   2022-04-23
677911   2022-04-23
127159   2022-04-23
Name: log_date, Length: 945518, dtype: datetime64[ns]